## Prozkoumejte shlukování metodou K-Means pomocí R a principů tidy dat.

### [**Kvíz před lekcí**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

V této lekci se naučíte, jak vytvářet shluky pomocí balíčku Tidymodels a dalších balíčků z ekosystému R (říkejme jim přátelé 🧑‍🤝‍🧑) a nigerijského hudebního datasetu, který jste si dříve importovali. Probereme základy metody K-Means pro shlukování. Mějte na paměti, že jak jste se naučili v předchozí lekci, existuje mnoho způsobů, jak pracovat se shluky, a metoda, kterou použijete, závisí na vašich datech. Vyzkoušíme metodu K-Means, protože je to nejběžnější technika shlukování. Pojďme začít!

Pojmy, o kterých se dozvíte:

-   Silhouette skóre

-   Metoda lokte

-   Inerční moment

-   Variance

### **Úvod**

[Shlukování metodou K-Means](https://wikipedia.org/wiki/K-means_clustering) je metoda pocházející z oblasti zpracování signálů. Používá se k rozdělení a seskupení dat do `k shluků` na základě podobností jejich vlastností.

Shluky lze vizualizovat jako [Voronoiovy diagramy](https://wikipedia.org/wiki/Voronoi_diagram), které zahrnují bod (nebo „semínko“) a jeho odpovídající oblast.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika od Jen Looper</figcaption>


Shlukování metodou K-Means zahrnuje následující kroky:

1.  Datový vědec nejprve určí požadovaný počet shluků, které mají být vytvořeny.

2.  Algoritmus poté náhodně vybere K pozorování z datové sady, která budou sloužit jako počáteční středy shluků (tj. centroidy).

3.  Každé z ostatních pozorování je následně přiřazeno k nejbližšímu centroidu.

4.  Poté se vypočítá nový průměr každého shluku a centroid se přesune na tento průměr.

5.  Jakmile jsou středy přepočítány, každé pozorování se znovu zkontroluje, zda by nemohlo být blíže jinému shluku. Všechna pozorování jsou znovu přiřazena pomocí aktualizovaných průměrů shluků. Kroky přiřazení shluků a aktualizace centroidů se iterativně opakují, dokud se přiřazení shluků nepřestane měnit (tj. dokud není dosaženo konvergence). Algoritmus obvykle končí, když každá nová iterace vede k zanedbatelnému pohybu centroidů a shluky se stanou statickými.

<div>

> Všimněte si, že kvůli náhodnosti počátečních k pozorování použitých jako výchozí centroidy můžeme pokaždé, když aplikujeme tento postup, získat mírně odlišné výsledky. Z tohoto důvodu většina algoritmů používá několik *náhodných startů* a vybírá iteraci s nejnižší hodnotou WCSS. Proto se důrazně doporučuje vždy spouštět metodu K-Means s několika hodnotami *nstart*, aby se předešlo *nežádoucímu lokálnímu optimu.*

</div>

Tato krátká animace využívající [ilustrace](https://github.com/allisonhorst/stats-illustrations) od Allison Horst vysvětluje proces shlukování:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>



Základní otázka, která při shlukování vyvstává, zní: jak zjistíte, na kolik shluků máte svá data rozdělit? Jednou z nevýhod metody K-Means je skutečnost, že musíte určit `k`, tedy počet `centroidů`. Naštěstí vám `metoda lokte` pomůže odhadnout dobrý výchozí počet `k`. Za chvíli si to vyzkoušíte.

### 

**Předpoklady**

Navážeme přesně tam, kde jsme skončili v [předchozí lekci](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), kde jsme analyzovali datovou sadu, vytvořili spoustu vizualizací a filtrovali datovou sadu na pozorování, která nás zajímají. Určitě si ji projděte!

Budeme potřebovat několik balíčků, abychom mohli začít s tímto modulem. Můžete si je nainstalovat pomocí: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativně níže uvedený skript zkontroluje, zda máte balíčky potřebné k dokončení tohoto modulu, a v případě, že některé chybí, je pro vás nainstaluje.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Pojďme rovnou na věc!

## 1. Tanec s daty: Zúžení na 3 nejoblíbenější hudební žánry

Toto je rekapitulace toho, co jsme dělali v předchozí lekci. Pojďme si trochu pohrát s daty!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 To šlo dobře!

## 2. Další průzkum dat.

Jak čistá jsou tato data? Podívejme se na odlehlé hodnoty pomocí krabicových grafů. Zaměříme se na číselné sloupce s menším počtem odlehlých hodnot (i když byste mohli odlehlé hodnoty odstranit). Krabicové grafy mohou ukázat rozsah dat a pomohou vybrat, které sloupce použít. Všimněte si, že krabicové grafy neukazují rozptyl, což je důležitý prvek pro dobře seskupitelná data. Další informace naleznete v [této diskusi](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Krabicové grafy](https://en.wikipedia.org/wiki/Box_plot) se používají k grafickému znázornění rozložení `číselných` dat, takže začněme *výběrem* všech číselných sloupců spolu s oblíbenými hudebními žánry.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Podívejte se, jak výběrový pomocník `where` usnadňuje tento proces 💁? Prozkoumejte další podobné funkce [zde](https://tidyselect.r-lib.org/).

Protože budeme vytvářet krabicové grafy pro každou číselnou vlastnost a chceme se vyhnout použití smyček, přeformátujeme naše data do *delšího* formátu, který nám umožní využít `facets` - podgrafy, z nichž každý zobrazuje jednu podmnožinu dat.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Mnohem delší! Teď je čas na nějaké `ggplots`! Tak jaký `geom` použijeme?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Teď můžeme vidět, že tato data jsou trochu hlučná: při pohledu na jednotlivé sloupce jako boxploty můžete vidět odlehlé hodnoty. Mohli byste projít dataset a tyto odlehlé hodnoty odstranit, ale to by data značně zredukovalo.

Prozatím si vybereme, které sloupce použijeme pro náš cvičný úkol shlukování. Vybereme číselné sloupce s podobnými rozsahy. Mohli bychom zakódovat `artist_top_genre` jako číselné hodnoty, ale prozatím ho vynecháme.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Výpočet k-means shlukování v R

K-means můžeme vypočítat v R pomocí vestavěné funkce `kmeans`, viz `help("kmeans()")`. Funkce `kmeans()` přijímá datový rámec, který obsahuje pouze číselné sloupce, jako svůj hlavní argument.

Prvním krokem při použití k-means shlukování je specifikace počtu shluků (k), které budou vytvořeny ve finálním řešení. Víme, že máme 3 hudební žánry, které jsme vyčlenili z datasetu, takže zkusme 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Objekt kmeans obsahuje několik informací, které jsou dobře vysvětleny v `help("kmeans()")`. Prozatím se zaměříme na několik z nich. Vidíme, že data byla rozdělena do 3 klastrů o velikostech 65, 110, 111. Výstup také obsahuje středové body (průměry) pro 3 skupiny napříč 5 proměnnými.

Vektor klastrů představuje přiřazení klastru pro každé pozorování. Použijme funkci `augment`, abychom přidali přiřazení klastru do původního datového souboru.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfektní, právě jsme rozdělili náš datový soubor do 3 skupin. Takže, jak dobré je naše shlukování 🤷? Podívejme se na `Silhouette skóre`.

### **Silhouette skóre**

[Silhouette analýza](https://en.wikipedia.org/wiki/Silhouette_(clustering)) může být použita ke studiu vzdálenosti mezi výslednými shluky. Toto skóre se pohybuje od -1 do 1, a pokud je skóre blízko 1, shluk je hustý a dobře oddělený od ostatních shluků. Hodnota blízko 0 představuje překrývající se shluky se vzorky velmi blízko rozhodovací hranice sousedních shluků. [zdroj](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metoda průměrného silhouette skóre počítá průměrné silhouette skóre pozorování pro různé hodnoty *k*. Vysoké průměrné silhouette skóre naznačuje dobré shlukování.

Funkce `silhouette` v balíčku cluster slouží k výpočtu průměrné šířky silhouette.

> Silhouette lze vypočítat pomocí jakékoli [vzdálenostní](https://en.wikipedia.org/wiki/Distance "Distance") metriky, jako je [Euklidovská vzdálenost](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") nebo [Manhattanská vzdálenost](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), které jsme probírali v [předchozí lekci](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Náš skóre je **.549**, tedy přesně uprostřed. To naznačuje, že naše data nejsou obzvlášť vhodná pro tento typ shlukování. Podívejme se, zda můžeme tento předpoklad vizuálně potvrdit. Balíček [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) poskytuje funkce (`fviz_cluster()`) pro vizualizaci shlukování.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Překrývání v klastrech naznačuje, že naše data nejsou příliš vhodná pro tento typ klastrování, ale pojďme pokračovat.

## 4. Určení optimálního počtu klastrů

Základní otázka, která často vyvstává při klastrování metodou K-Means, je tato: bez známých třídních štítků, jak zjistíte, na kolik klastrů rozdělit svá data?

Jedním ze způsobů, jak to zjistit, je použít vzorek dat k `vytvoření série modelů klastrování` s postupně se zvyšujícím počtem klastrů (např. od 1 do 10) a vyhodnotit metriky klastrování, jako je **Silhouette skóre.**

Určeme optimální počet klastrů výpočtem algoritmu klastrování pro různé hodnoty *k* a vyhodnocením **součtu čtverců uvnitř klastrů** (WCSS). Celkový součet čtverců uvnitř klastrů (WCSS) měří kompaktnost klastrování a chceme, aby byl co nejmenší, přičemž nižší hodnoty znamenají, že datové body jsou blíže u sebe.

Prozkoumejme vliv různých voleb hodnoty `k`, od 1 do 10, na toto klastrování.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nyní, když máme celkový součet čtverců uvnitř klastrů (tot.withinss) pro každý algoritmus shlukování s centrem *k*, použijeme [metodu lokte](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) k nalezení optimálního počtu klastrů. Tato metoda spočívá v zakreslení WCSS jako funkce počtu klastrů a výběru [lokte křivky](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") jako počtu klastrů, které použijeme.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Graf ukazuje výrazné snížení WCSS (tedy větší *soudržnost*), když se počet klastrů zvýší z jednoho na dva, a další znatelné snížení při přechodu ze dvou na tři klastry. Poté je snížení méně výrazné, což vede k vytvoření `lokte` 💪 na grafu přibližně u tří klastrů. To je dobrý náznak, že existují dva až tři poměrně dobře oddělené klastry datových bodů.

Nyní můžeme přistoupit k extrakci modelu klastrování, kde `k = 3`:

> `pull()`: používá se k extrakci jednoho sloupce
>
> `pluck()`: používá se k indexování datových struktur, jako jsou seznamy


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Skvělé! Pojďme si vizualizovat získané clustery. Máte chuť na trochu interaktivity s použitím `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Možná bychom očekávali, že každý klastr (reprezentovaný různými barvami) bude mít odlišné žánry (reprezentované různými tvary).

Podívejme se na přesnost modelu.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Přesnost tohoto modelu není špatná, ale ani skvělá. Může to být způsobeno tím, že data nejsou vhodná pro K-Means Clustering. Tato data jsou příliš nevyvážená, málo korelovaná a mezi hodnotami sloupců je příliš velká variabilita, což ztěžuje jejich seskupování. Ve skutečnosti jsou vytvořené clustery pravděpodobně silně ovlivněny nebo zkresleny třemi kategoriemi žánrů, které jsme výše definovali.

Nicméně, i tak to byl skvělý proces učení!

V dokumentaci Scikit-learn můžete vidět, že model jako tento, s clustery, které nejsou příliš dobře vymezené, má problém s „variabilitou“:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika ze Scikit-learn</figcaption>



## **Variabilita**

Variabilita je definována jako „průměr čtverců odchylek od průměru“ [zdroj](https://www.mathsisfun.com/data/standard-deviation.html). V kontextu tohoto problému seskupování to znamená, že hodnoty v našem datasetu mají tendenci se od průměru odchylovat až příliš.

✅ Toto je skvělý moment k zamyšlení nad všemi způsoby, jak tento problém vyřešit. Můžete data ještě více upravit? Použít jiné sloupce? Nebo jiný algoritmus? Tip: Zkuste [škálovat svá data](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), abyste je normalizovali, a otestujte jiné sloupce.

> Vyzkoušejte tento '[kalkulátor variability](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', abyste lépe pochopili tento koncept.

------------------------------------------------------------------------

## **🚀Výzva**

Stravte nějaký čas s tímto notebookem a upravujte parametry. Dokážete zlepšit přesnost modelu tím, že data více vyčistíte (například odstraněním odlehlých hodnot)? Můžete použít váhy, abyste některým vzorkům dat přidali větší důležitost. Co dalšího můžete udělat, abyste vytvořili lepší clustery?

Tip: Zkuste škálovat svá data. V notebooku je zakomentovaný kód, který přidává standardní škálování, aby se hodnoty sloupců více podobaly z hlediska rozsahu. Zjistíte, že i když skóre siluety klesne, „zlom“ v grafu lokte se vyhladí. To je proto, že ponechání dat neškálovaných umožňuje, aby data s menší variabilitou měla větší váhu. Přečtěte si o tomto problému více [zde](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kvíz po přednášce**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Opakování a samostudium**

-   Podívejte se na simulátor K-Means [například tento](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Tento nástroj můžete použít k vizualizaci vzorků datových bodů a určení jejich centroidů. Můžete upravit náhodnost dat, počet clusterů a počet centroidů. Pomáhá vám to získat představu o tom, jak lze data seskupit?

-   Podívejte se také na [tento materiál o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) ze Stanfordu.

Chcete si vyzkoušet své nově nabyté dovednosti v seskupování na datasetech, které se dobře hodí pro K-Means clustering? Podívejte se na:

-   [Trénování a hodnocení modelů seskupování](https://rpubs.com/eR_ic/clustering) pomocí Tidymodels a dalších nástrojů

-   [Analýza clusterů pomocí K-means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering s principy tidy dat](https://www.tidymodels.org/learn/statistics/k-means/)

## **Úkol**

[Vyzkoušejte různé metody seskupování](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## PODĚKOVÁNÍ:

[Jen Looper](https://www.twitter.com/jenlooper) za vytvoření původní verze tohoto modulu v Pythonu ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za vytvoření úžasných ilustrací, které dělají R přívětivějším a poutavějším. Další ilustrace najdete v její [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Přejeme příjemné učení,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
